# MultiProcessing

- more speed 
- more awsome

https://docs.python.org/ko/3/library/multiprocessing.html

- Python multiprocessing Tool
    - https://docs.python.org/ko/3/library/multiprocessing.html
    
#### mulitprocessing using Process Spawning & use Pool Object
- Process Spawning
    - parent process request make child process 

### List Comprehension VS MuliProcessing(python)

In [ ]:
import numpy as np
from multiprocessing import Pool
pool = Pool(4)

arr = np.random.random(1000000)

def mul(x):
    return x * 10

In [ ]:
%time
# Serial Python
result = [mul(x) for x in arr]

In [ ]:
%time
with Pool(processes=4) as p:
    result = p.map(mul, arr)
    Pool.close()

#### Multiprocessing 빠른거라며.. 왜 오래걸려..
- multiprocessing 라이브럴리 동작 방식
    - 큰데이터를 다른 프로세스로 전달할 때 pickle을 이용 Serialize 한 뒤 전달
    - n개의 프로세스에 작업하도록 설정했음으로 데이터 복사본을 n개 만큼 만들어야 함으로 큰 메모리 사용
    - 역 직렬화 과정을 통해 데이터를 받아야 함으로 큰 오버헤드 발생
    - 선언한 프로세스 풀을 닫지 않고 방치하면 프로세스가 메모리에 잔존 (memory leakge)
    - 위 현상을 방지 하기 위해 contextlib 라이블러리의 closing() 함수 사용

#### 정리
- 프로세스에 데이터 전달하는 행위가 많아짐으로 
- 수행 시간에 악영향
- 조금 더 복잡한 연산에서는 multiprocessing이 빠른 속도를 보이지만 
- 메모리 문제는 처리 불가.. 

> 직렬화
- 데이터 구조나 오브젝트 상태를 동일하거나 다른 컴퓨터 환경에 저장하고 나중에 재구성할 수 있는 포맷으로 변환하는 과정이다. 
- 오브젝트를 직렬화하는 과정은 오브젝트를 마샬링

> 오버헤드
- 어떤 처리를 하기 위해 들어가는 간접적인 처리 시간 · 메모리

# RAY 등장
- 분산 app을 위해 단순하고 범용적인 api 제공

### 장점
- 병렬 처리를 위해 코드 고칠 필요 없음
- 직렬화 오버헤드 문제가 발생 안함
- 빠름!!

In [ ]:
# pip install ray

In [ ]:
import os 
num_cpu = os.cpu_count()
print('num_cpu :', num_cpu)

In [ ]:
import ray
import numpy as np

ray.init(num_cpus=num_cpu)

@ray.remote
def mul(x):
    return x * 10

arr = np.random.random(1000000)
arr = ray.put(arr)

In [ ]:
%time
result = ray.get(mul.remote(arr))

### 사용법
- 기존 함수에 Decorator 추가
- 함수 호출할때 remote 메서드 사용!
- 빨라짐!!!

#### 왜 빠른데..
- Ray 직렬화 오버헤드 해결 위해
- Apache Arrow를 사용
- 행 기반이 아닌 컬럼 기반의 인메모리 포멧
- Zero-copy 직렬화 수행
- 직렬화된 데이터를 인메모리 객체 저장소인 Plasma를 이용 빠르게 직렬화된 데이터 공유 

# more cpu more speed!!

<img src="https://media.vlpt.us/images/otzslayer/post/49eb637c-add1-4030-b46d-d4c90800c76c/ray_benchmark.png">

## Ray 철학 
- 둘 이상의 머신에서 동일한 코드 실행 (쿠버네티스 Ray)
- 상호 통신 가능한 microservice & actor 
- 장애 대응 적절히(dashboard)
- 큰 객체 수치 데이터 효율적으로 다룰수 있어야함


<img src= "https://media.vlpt.us/images/otzslayer/post/018fb163-fa86-4f95-80b3-ca65114b7a12/ray_ecosystem.png">

# Ray 구성 
- Task
- Object
- Actor

# Task 
- 호출하는 곳과 다른 프로세스에서 실행되는 함수 or class
- 함수를 @ray.remote라는 데코레이터로 감쌌다면 그 함수를 Task
- Remote Function이라고도 불르며, 호출자와 비동기적(asynchronously)으로 실행됨 
- remote() 매서드를 써서 호출 가능 objectRef라는 값 반환
- ray.get(objectRef)하여 Task 실행하고 값 반환 받을 수 있다.

> Asynchronously 
- 통신에서 비동기식 통신은 일반적으로 외부 클록 신호를 사용하지 않고 데이터를 전송하는 것으로, 데이터가 안정적인 스트림이 아닌 간헐적으로 전송될 수 있습니다. 통신 심볼에서 데이터를 복구하는 데 필요한 모든 타이밍은 심볼 내에 인코딩됩니다. 위 

# Object
- Task를 통해 반환되거나, ray.put()을 통해 생성되는 값
- 데이터의 크기가 큰 경우 ray.put()을 통해 Object로 만들어 
- Ray에서 빠르게 사용할수 있습니다.
- Object는 Spark에 RDD 같이 immutable 합니다

# Actor
- Stateful한 워커 프로세스
- @ray.remote를 통해 Actor Class 만들수 있고 
- Actor 클래스의 메서드 호출은 Statefull Task 입니다.

> Statefull
- Stateful '구조'는 Server와 Client간 세션의 'State(상태)'에 기반하여 Client에 response를 보냅니다. 
- 이를 위해 세션 '상태'를 포함한 Client와의 세션 정보를 server에 저장하게 됩니다. 



## Ray 사용법

In [3]:
import ray

In [6]:
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-01-12_10-43-25_447418_9282/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-12_10-43-25_447418_9282/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-12_10-43-25_447418_9282',
 'metrics_export_port': 58974,
 'node_id': '12e3fc62534a8dcc0070b9ed9f50521a1ad9909c31622c4748e922ca'}

In [11]:
# 큰 행렬 두개 만들어 Dot Product 실행
import numpy as np

def create_matrix(size):
    return np.random.normal(size=size)

def multiply_matrices(x, y):
    return np.dot(x, y)

In [12]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)


### Remote Function 완료 

In [14]:
x_id = create_matrix.remote([1000, 1000])
y_id = create_matrix.remote([1000, 1000])
z_id = multiply_matrices.remote(x_id, y_id)

In [15]:
z = ray.get(z_id)

In [16]:
ray.shutdown()

In [17]:
z

array([[-39.60062485,  15.78877369,  16.68144368, ...,  11.78802126,
        -53.32911563, -64.82382413],
       [-15.44164969,  29.83216709,  62.12148908, ...,   4.91482283,
         60.64081446, -19.45758429],
       [-26.47200579, -11.84023034, -14.42046045, ..., -12.10056708,
         47.68924596, -31.25880392],
       ...,
       [-34.22715986, -62.55700455,  52.20114497, ...,   6.8668783 ,
          5.21971249, -53.72344791],
       [ -2.26272891,  17.57374541,  42.41414844, ..., -35.85465069,
        -18.41274374,   1.55255334],
       [ -7.10775999,  25.16403206,  78.31260414, ..., -29.1606261 ,
          4.5641617 ,  15.46001114]])

# Tips 
- ray.put() 
- 큰 데이터를 반복적으로 사용하게 된다면 ray.put()을 사용
- 메모리 사용 줄일수 있다.
- ray.put()은 데이터를 공유 메모리에 저장
- 복사본을 만들지 않고 모든 프로세스에서 접근할수 있다 

- 크기가 작은 작업은 Ray의 Task로 만들지 않는다
- 스케줄링이나 내부 커뮤니케이션으로 불필요한 오버헤드 발생.. 
- 큰 Tsk 생성해서 작은 함수를 여러번 실행하는게 더 좋을수도 

- Tensorflow와 함께 사용할때 Pickling 이슈 발생
- Remote Function 안에서 import tensorflow as tf 호출하여 해결할 수 있게 해준다